In [1]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
%matplotlib inline

from datetime import date,datetime,timedelta
from dateutil.parser import parse

import json
import re
from glob import glob

import seaborn as sns
import folium as g

import matplotlib.pyplot as plt
from matplotlib import rc
# 한글 폰트 문제 해결
rc('font', family='AppleGothic')
# 차트 축 <- 음수 부호 지원
plt.rcParams['axes.unicode_minus'] = False

from bs4 import BeautifulSoup
from urllib.request import urlopen
from urllib.error   import HTTPError
from urllib.error   import URLError

import requests
import csv

In [16]:
f = open('fifa.csv','w', newline='')
wr = csv.writer(f)

wr.writerow(['Id','Name','Age','Nationality','Team','Contract','Height','Weight','Preferred Foot',
             'Overall','Potential','Best overall','Best position','Growth','Joined','Loan date end','Value','Wage','Total Stats',
             'Base Stats','International Reputation'])

for offset in range(0,319):
    url = 'https://sofifa.com/players?type=all&r=220001&set=true&showCol%5B%5D=pi&showCol%5B%5D=ae&showCol%5B%5D=hi&showCol%5B%5D=wi&showCol%5B%5D=pf&showCol%5B%5D=oa&showCol%5B%5D=pt&showCol%5B%5D=bo&showCol%5B%5D=bp&showCol%5B%5D=gu&showCol%5B%5D=jt&showCol%5B%5D=le&showCol%5B%5D=vl&showCol%5B%5D=wg&showCol%5B%5D=tt&showCol%5B%5D=bs&showCol%5B%5D=ir&offset='
    url += str(offset*60) 
    
    webpage = requests.get(url, headers={"User-Agent": "Mozilla/5.0"})
    soup = BeautifulSoup(webpage.text,'html.parser')
    trs = soup.select('tbody>tr')
    for tr in trs:
        player_id = tr.select('td.col-pi')[0].text
        name = tr.select('.ellipsis')[0].text
        team = tr.select('td.col-name')[1].select('a')[0].text
        contract = tr.select('td.col-name')[1].select('.sub')[0].text
        nationality = tr.select('td.col-name > img')[0]['title']
        age = tr.select('td.col.col-ae')[0].text
        height = tr.select('td.col.col-hi')[0].text
        weight = tr.select('td.col.col-wi')[0].text
        foot = tr.select('td.col.col-pf')[0].text
        ova = tr.select('td.col.col-oa')[0].text
        best_overall = tr.select('td.col.col-bo')[0].text
        potential = tr.select('td.col.col-pt')[0].text
        best_position = tr.select('td.col.col-bp')[0].text
        growth = tr.select('td.col.col-gu')[0].text
        joined = tr.select('td.col.col-jt')[0].text
        loan_ended = tr.select('td.col.col-le')[0].text
        value = tr.select('td.col.col-vl')[0].text
        wage = tr.select('td.col.col-wg')[0].text
        total_stat = tr.select('td.col.col-tt')[0].text
        base_stat = tr.select('td.col.col-bs')[0].text
        international_ranking = tr.select('td.col.col-ir')[0].text
        wr.writerow([player_id,name,age,nationality,team,contract,height,
                    weight,foot,ova,potential,best_overall,best_position,growth,joined,
                    loan_ended,value,wage,total_stat,base_stat,international_ranking])
f.close()

In [2]:
fifa = pd.read_csv('fifa.csv')

In [3]:
## 이게 찐

url_list = {'Germany':'https://sofifa.com/team/21/fc-bayern-munchen/live?r=210064&set=true',
            'England':'https://sofifa.com/team/1/arsenal/live?r=210064&set=true',
            'Spain':'https://sofifa.com/team/241/fc-barcelona/live?r=210064&set=true',
            'Italy':'https://sofifa.com/team/47/ac-milan/live?r=210064&set=true',
            'France':'https://sofifa.com/team/73/paris-saint-germain/live?r=210064&set=true'}

league_table = {}

for league,url in url_list.items():
    print(url)
    webpage = requests.get(url, headers={'User-Agent':'Chrome/66.0.3359.181'})
    tables = pd.read_html(webpage.content,encoding='UTF-8')
    league_table[league] = tables[0]
    
def clean_league_table(x):
    table = x.droplevel(level=0,axis=1)
    table.columns = range(table.columns.size)
    table = table[[0,2,3,4,6,7,8,9]]
    table.columns = ['Rank','Team','Points','GD','Match','Win','Draw','Loss']
    table = table.set_index('Rank')
    return table

Germany = clean_league_table(league_table['Germany'])
England = clean_league_table(league_table['England'])
Spain = clean_league_table(league_table['Spain'])
Italy = clean_league_table(league_table['Italy'])
France = clean_league_table(league_table['France'])


https://sofifa.com/team/21/fc-bayern-munchen/live?r=210064&set=true
https://sofifa.com/team/1/arsenal/live?r=210064&set=true
https://sofifa.com/team/241/fc-barcelona/live?r=210064&set=true
https://sofifa.com/team/47/ac-milan/live?r=210064&set=true
https://sofifa.com/team/73/paris-saint-germain/live?r=210064&set=true


In [4]:
def check_team_name(x):
    league_list = [Germany,England,France,Spain,Italy]
    for league in league_list:
        for name in league['Team'].values:
            result = (x['Team']==name).sum()
            if result==0:
                print(name,result)
            
check_team_name(fifa)      

In [5]:
def find_league(x):
    if x in Germany['Team'].to_list():
        return 'Germany'
    elif x in England['Team'].to_list():
        return 'England'
    elif x in France['Team'].to_list():
        return 'France'
    elif x in Spain['Team'].to_list():
        return 'Spain'
    elif x in Italy['Team'].to_list():
        return 'Italy'
    else: 
        return 'Else'

# add league column    
fifa['League'] = fifa['Team'].apply(find_league)
fifa['League']

0         France
1        Germany
2         France
3          Spain
4        England
          ...   
19093       Else
19094       Else
19095       Else
19096       Else
19097       Else
Name: League, Length: 19098, dtype: object

In [6]:
# Joined
fifa['Joined'] = pd.to_datetime(fifa['Joined'],format='%b %d, %Y')

# height
fifa['Height'] = fifa['Height'].str.replace('cm','').astype(int)

# weight
fifa['Weight'] = fifa['Weight'].str.replace('kg','').astype(int)

# contract split
# free contract or loan => identified as 0
fifa['Contract'] = fifa['Contract'].str.replace('\n','')

def contract_start(x):
    if x.startswith('2'):
        return x[0:5]
    else: return 0

def contract_end(x):
    if x.startswith('2'):
        return x[6:11]
    else: return 0

fifa['Contract_start'] = fifa['Contract'].apply(contract_start)
fifa['Contract_end'] = fifa['Contract'].apply(contract_end)
fifa['Free contract'] = fifa['Contract'].str.startswith('Free')
fifa.drop(['Contract'],axis=1,inplace=True)

# Loan date
fifa['Loan date end'] = pd.to_datetime(fifa['Loan date end'],format='%b %d, %Y')

# value 
def calc_value(x):
    if x[-1] == 'M':
        return float(x[:-1])*1000000
    elif x[-1] =='K':
        return float(x[:-1])*1000
    else:
        return float(x)
    
fifa['Value'] = fifa['Value'].str.extract(r'(\d+\.?\d?K?M?)')[0].apply(calc_value)

# wage
def calc_wage(x):
    if x[-1] == 'K':
        return float(x[:-1])*1000
    else:
        return float(x)
fifa['Wage'] = fifa['Wage'].str.extract(r'(\d+\.?\d?K?)')[0].apply(calc_wage)

In [7]:
display(fifa)

,Id,Name,Age,Nationality,Team,Height,Weight,Preferred Foot,Overall,Potential,Best overall,Best position,Growth,Joined,Loan date end,Value,Wage,Total Stats,Base Stats,International Reputation,League,Contract_start,Contract_end,Free contract
0,158023,L. Messi,34,Argentina,Paris Saint-Germain,170,72,Left,93,93,93,RW,0,2021-08-10,NaT,78000000.0,320000.0,2219,462,5,France,2021,2023,False
1,188545,R. Lewandowski,32,Poland,FC Bayern München,185,81,Right,92,92,92,ST,0,2014-07-01,NaT,119500000.0,270000.0,2212,460,5,Germany,2014,2023,False
2,231747,K. Mbappé,22,France,Paris Saint-Germain,182,73,Right,91,95,92,ST,4,2018-07-01,NaT,194000000.0,230000.0,2175,470,4,France,2018,2022,False
3,200389,J. Oblak,28,Slovenia,Atlético de Madrid,188,87,Right,91,93,91,GK,2,2014-07-16,NaT,112000000.0,130000.0,1413,489,5,Spain,2014,2023,False
4,192985,K. De Bruyne,30,Belgium,Manchester City,181,70,Right,91,91,91,CM,0,2015-08-30,NaT,125500000.0,350000.0,2304,485,4,England,2015,2025,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19093,256414,P. Ferry,18,Northern Ireland,Derry City,179,73,Right,47,59,49,ST,12,2020-01-01,NaT,110000.0,500.0,1176,233,1,Else,2020,2021,False
19094,262820,L. Rudden,19,Republic of Ireland,Finn Harps,173,66,Right,47,60,49,ST,13,2021-06-18,NaT,110000.0,500.0,1210,255,1,Else,2021,2021,False
19095,261547,R. By,22,India,Chennaiyin FC,181,74,Right,47,55,47,GK,8,2020-10-28,NaT,90000.0,500.0,811,259,1,Else,2020,2022,False
19096,261818,R. Gallagher,20,Republic of Ireland,Finn Harps,180,74,Right,47,61,49,CAM,14,2018-01-01,NaT,110000.0,500.0,1255,269,1,Else,2018,2021,False


# 시각화를 어떤 방향으로 해야할까